In [1]:
#model套件
import torch
from torch import nn as nn
#資料處理套件
import numpy as np
#時間套件
import time
#列舉套件
from enum import Enum

import sys
#將父目錄添加到模組搜尋路徑中
sys.path.append('..')
#取得模型
import GetModel as GM
#訓練/測試 模型
import TrainModel as TM
#資料前處理
import TrainDataPreprocessing as TDP

In [2]:
#存檔位置
ModelMode = "ESC10_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
sMode = 0
filePath_Train = ModelMode + TrainDataSoundMode[sMode] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[sMode] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[sMode]

In [3]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
data_1S = TDP.loadDataESC10(soundLength=1)
print("讀取3秒資料")
data_3S = TDP.loadDataESC10(soundLength=3)
print("讀取5秒資料")
data_5S = TDP.loadDataESC10(soundLength=5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料
讀取3秒資料
讀取5秒資料
Data Loading Time: 0.05 seconds


In [4]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []


#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    print("----------------------------------------------------------------------------------------------------------------")
    print("fold: ", fold)
    print("----------------------------------------------------------------------------------------------------------------")

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = GM.getModel(num_classes=10)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    #------------------------------------------------
    #需要調整的地方
    #------------------------------------------------
    #學習率
    lr= 0.01
    #epoch
    epoch= 90
    #訓練資料(要用幾秒訓練) data_1S/data_3S/data_5S
    trainData = data_1S
    #------------------------------------------------
    
    #紀錄訓練開始時間
    start_time = time.time()

    #訓練模型
    model, train_loss, train_accuracy = TM.train_model_TrainAll(model=model,
                                                                inputData=trainData,
                                                                trainFold=fold,
                                                                setLR=lr,
                                                                setEpoch=epoch)
    
    #測試準確度
    test_accuracy1 = TM.test_model(model=model, inputData=data_1S, testFold=fold)
    test_accuracy3 = TM.test_model(model=model, inputData=data_3S, testFold=fold)
    test_accuracy5 = TM.test_model(model=model, inputData=data_5S, testFold=fold)

    #紀錄訓練結束時間
    end_time = time.time()

    #計算訓練耗時
    training_time = end_time - start_time

    #印出訓練耗時
    print("Training Time: {:.2f} seconds".format(training_time))

    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("訓練過程\n")
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    for i in range(len(train_loss)):
        # 寫入資料
        f.write("Fold: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
    f.write("Training Time: {:.2f} seconds\n".format(training_time))
    f.write("---------------------------------------------------------------------------------------------------------------\n")

    #--------------------------------------------------------------------

    
    kFoldTrainTime.append(training_time)
    kFoldTrainAccuracy.append(train_accuracy[-1])
    kFoldTestAccuracy1.append(test_accuracy1)
    kFoldTestAccuracy3.append(test_accuracy3)
    kFoldTestAccuracy5.append(test_accuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

----------------------------------------------------------------------------------------------------------------
fold:  1
----------------------------------------------------------------------------------------------------------------
可用的 GPU 数量: 1
Train Epoch: 1/90 Train_Loss: 1.2540373802185059 Train_acc: 36.750000%
Train Epoch: 2/90 Train_Loss: 0.8135437369346619 Train_acc: 63.937500%
Train Epoch: 3/90 Train_Loss: 0.7292293906211853 Train_acc: 76.312500%
Train Epoch: 4/90 Train_Loss: 0.31030774116516113 Train_acc: 83.375000%
Train Epoch: 5/90 Train_Loss: 0.38556069135665894 Train_acc: 85.187500%
Train Epoch: 6/90 Train_Loss: 0.33097416162490845 Train_acc: 88.062500%
Train Epoch: 7/90 Train_Loss: 0.23989444971084595 Train_acc: 89.187500%
Train Epoch: 8/90 Train_Loss: 0.27434512972831726 Train_acc: 90.687500%
Train Epoch: 9/90 Train_Loss: 0.20509454607963562 Train_acc: 91.187500%
Train Epoch: 10/90 Train_Loss: 0.22852744162082672 Train_acc: 91.250000%
Train Epoch: 11/90 Train_Loss: 0.

Train Epoch: 15/90 Train_Loss: 0.1619657427072525 Train_acc: 93.562500%
Train Epoch: 16/90 Train_Loss: 0.19948720932006836 Train_acc: 94.250000%
Train Epoch: 17/90 Train_Loss: 0.46669650077819824 Train_acc: 93.125000%
Train Epoch: 18/90 Train_Loss: 0.18535767495632172 Train_acc: 93.187500%
Train Epoch: 19/90 Train_Loss: 0.06952912360429764 Train_acc: 94.062500%
Train Epoch: 20/90 Train_Loss: 0.19486403465270996 Train_acc: 93.125000%
Train Epoch: 21/90 Train_Loss: 0.2274019420146942 Train_acc: 93.437500%
Train Epoch: 22/90 Train_Loss: 0.19767844676971436 Train_acc: 93.875000%
Train Epoch: 23/90 Train_Loss: 0.09630739688873291 Train_acc: 94.062500%
Train Epoch: 24/90 Train_Loss: 0.14832396805286407 Train_acc: 94.437500%
Train Epoch: 25/90 Train_Loss: 0.08753374218940735 Train_acc: 93.625000%
Train Epoch: 26/90 Train_Loss: 0.062058642506599426 Train_acc: 94.062500%
Train Epoch: 27/90 Train_Loss: 0.06838478893041611 Train_acc: 93.125000%
Train Epoch: 28/90 Train_Loss: 0.09489041566848755 T

Train Epoch: 33/90 Train_Loss: 0.17418113350868225 Train_acc: 95.000000%
Train Epoch: 34/90 Train_Loss: 0.11660256236791611 Train_acc: 95.000000%
Train Epoch: 35/90 Train_Loss: 0.09833686798810959 Train_acc: 94.875000%
Train Epoch: 36/90 Train_Loss: 0.07582058757543564 Train_acc: 94.562500%
Train Epoch: 37/90 Train_Loss: 0.14228224754333496 Train_acc: 94.937500%
Train Epoch: 38/90 Train_Loss: 0.24270358681678772 Train_acc: 94.937500%
Train Epoch: 39/90 Train_Loss: 0.12977561354637146 Train_acc: 94.937500%
Train Epoch: 40/90 Train_Loss: 0.17140567302703857 Train_acc: 94.937500%
Train Epoch: 41/90 Train_Loss: 0.13503135740756989 Train_acc: 94.750000%
Train Epoch: 42/90 Train_Loss: 0.18803873658180237 Train_acc: 94.812500%
Train Epoch: 43/90 Train_Loss: 0.07245750725269318 Train_acc: 94.812500%
Train Epoch: 44/90 Train_Loss: 0.14729218184947968 Train_acc: 94.625000%
Train Epoch: 45/90 Train_Loss: 0.13101013004779816 Train_acc: 94.937500%
Train Epoch: 46/90 Train_Loss: 0.12532904744148254 

Train Epoch: 51/90 Train_Loss: 0.12213060259819031 Train_acc: 94.625000%
Train Epoch: 52/90 Train_Loss: 0.17640091478824615 Train_acc: 94.875000%
Train Epoch: 53/90 Train_Loss: 0.18609020113945007 Train_acc: 94.812500%
Train Epoch: 54/90 Train_Loss: 0.10080619901418686 Train_acc: 95.125000%
Train Epoch: 55/90 Train_Loss: 0.08930173516273499 Train_acc: 94.062500%
Train Epoch: 56/90 Train_Loss: 0.13782328367233276 Train_acc: 94.875000%
Train Epoch: 57/90 Train_Loss: 0.14753183722496033 Train_acc: 94.875000%
Train Epoch: 58/90 Train_Loss: 0.06402254849672318 Train_acc: 94.937500%
Train Epoch: 59/90 Train_Loss: 0.12613339722156525 Train_acc: 95.000000%
Train Epoch: 60/90 Train_Loss: 0.29832154512405396 Train_acc: 94.812500%
Train Epoch: 61/90 Train_Loss: 0.07188931852579117 Train_acc: 94.875000%
Train Epoch: 62/90 Train_Loss: 0.01502982247620821 Train_acc: 94.875000%
Train Epoch: 63/90 Train_Loss: 0.07956992089748383 Train_acc: 95.000000%
Train Epoch: 64/90 Train_Loss: 0.062052544206380844

Train Epoch: 69/90 Train_Loss: 0.08147785067558289 Train_acc: 94.500000%
Train Epoch: 70/90 Train_Loss: 0.1199374571442604 Train_acc: 94.375000%
Train Epoch: 71/90 Train_Loss: 0.18654125928878784 Train_acc: 94.562500%
Train Epoch: 72/90 Train_Loss: 0.15961450338363647 Train_acc: 94.375000%
Train Epoch: 73/90 Train_Loss: 0.19351111352443695 Train_acc: 94.000000%
Train Epoch: 74/90 Train_Loss: 0.28011733293533325 Train_acc: 94.500000%
Train Epoch: 75/90 Train_Loss: 0.16143472492694855 Train_acc: 94.500000%
Train Epoch: 76/90 Train_Loss: 0.0872170627117157 Train_acc: 94.250000%
Train Epoch: 77/90 Train_Loss: 0.10631710290908813 Train_acc: 94.812500%
Train Epoch: 78/90 Train_Loss: 0.07364210486412048 Train_acc: 94.500000%
Train Epoch: 79/90 Train_Loss: 0.06374813616275787 Train_acc: 94.625000%
Train Epoch: 80/90 Train_Loss: 0.12952719628810883 Train_acc: 94.437500%
Train Epoch: 81/90 Train_Loss: 0.186350479722023 Train_acc: 94.437500%
Train Epoch: 82/90 Train_Loss: 0.11926165223121643 Trai

In [5]:
#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    # 寫入資料
    f.write("Fold: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, kFoldTrainAccuracy[fold], kFoldTestAccuracy1[fold], kFoldTestAccuracy3[fold], kFoldTestAccuracy5[fold], kFoldTrainTime[fold]))
    f.write("-----------------------------------------------------------------------------------\n")
# 寫入資料
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(max(kFoldTrainAccuracy), max(kFoldTestAccuracy1), max(kFoldTestAccuracy3), max(kFoldTestAccuracy5)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(min(kFoldTrainAccuracy), min(kFoldTestAccuracy1), min(kFoldTestAccuracy3), min(kFoldTestAccuracy5)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(np.mean(kFoldTrainAccuracy), np.mean(kFoldTestAccuracy1), np.mean(kFoldTestAccuracy3), np.mean(kFoldTestAccuracy5)))

f.close()